# ITPS Flight Test Analysis Tool - v1.0
<p>Use this tool on FIREFOX or CHROME only </p>
<p>Safari and Edge have a few compatibility issues</p>

In [ ]:
# Data File Selection
from FTAT_imports import *

# make display 100% of the screen
display(HTML("<style>.container { width:100% !important; }</style>"))

# set this variable to True to bypass file import and generate fake data
unit_test = False

if not unit_test:
    f = FileBrowser()
    display(f.widget())  #
    #   <interact with widget, select a path>
    # in a separate cell:

In [ ]:
# Main Program


if unit_test:
    # creating fake data just to test
    rng = pd.date_range('27/10/2018 13:00:00', periods=5000, freq='50ms')
    raw_data = pd.DataFrame(data=rng, columns=['Time'])
    raw_data.set_index(['Time'], inplace=True)
    raw_data['counter'] = np.arange(len(raw_data))
    raw_data['fake_angle'] = raw_data.apply(lambda x: (raw_data['counter']/10)%(2*np.pi))
    raw_data['sin0'] = raw_data['counter'] #I have no idea why, but if I do not first create 'sine1' and then calculate with lambda, it throwns me an error
    raw_data['sin0'] = raw_data.apply(lambda x: np.sin(raw_data['fake_angle']))
    raw_data['cos0'] = raw_data['counter']
    raw_data['cos0'] = raw_data.apply(lambda x: np.cos(raw_data['fake_angle']))
    for i in range(30):
        sine_label = 'sin' + str(i+1)
        cosine_label = 'cos' + str(i+1)
        raw_data[sine_label] = raw_data['sin0']
        raw_data[cosine_label] = raw_data['cos0'] 
else:
    # select file and read from disk
    print('Preparing Graph Parameters ... may take a while ... patience is a virtue.')
    print('Reading data from {} ...'.format(f.path))
    #filepath='./IadsDataExport_clean.csv' #
    #raw_data=pd.read_csv(filepath, encoding='latin1', low_memory=False) #
    raw_data=pd.read_csv(f.path, encoding='latin1', low_memory=False) ##
    print('Done.')
    dirty_file = False
    if raw_data['Time'][10].count(':') > 2:
        dirty_file = True

    if dirty_file == True:
        ## FOR DIRTY DATA
        print('Dirty file detected ... cleaning up the data...')  #DIRTY DATA
        raw_data.fillna(value=0, inplace=True)  #DIRTY DATA
        raw_data['Time'] = (raw_data['Time'].str.slice_replace(0,4,''))  #DIRTY DATA

    raw_data['Time'] = pd.to_datetime(raw_data['Time'])   #CLEAN DATA
    raw_data = raw_data.set_index(['Time'])   #CLEAN DATA

print('Done.')
print('Imported data from: {} to: {}, with {} records'.format(raw_data.index[0], raw_data.index[-1], len(raw_data.index) ))

# Let's subsample to speed up the graphing process
desired_sub_sample_rate = 5 #Hz
counter = 0
start_sample = raw_data.index[0]
next_sample = start_sample
while (next_sample - start_sample <= np.timedelta64(1, 's')):
    counter = counter + 1
    next_sample = raw_data.index[counter]
if desired_sub_sample_rate <= counter:
    sub_sample_rate = int(counter / desired_sub_sample_rate)
else:
    sub_sample_rate = int(counter)
    desired_sub_sample_rate = counter
    
sub_sampled_data = raw_data[::sub_sample_rate]

print()
print('Original data at {} Hz'.format((counter-1)))
print('Data subsampled to {} Hz to speed up graphing. Reduced to {} records'.format(desired_sub_sample_rate, len(sub_sampled_data.index)))
print('Done.')



time_slices_db = {} #this is the time slices data base, TP#, Description, Start, Stop (indexes)
selected_slice = [] #this is a global variable to store the currently selected slice that will be stored in the time_slices_db


######################### Slice Map and Definitions ##################################################
slicemap = ParameterMap(sub_sampled_data, 'MediumSeaGreen', 'Parameter Map')



######################### Slice Graph Definitions ####################################################

# data
current_plot_data = sub_sampled_data


slice_plot = LinePlotBrush(current_plot_data.index.values, current_plot_data.index.values)
detail_plot = LinePlot(current_plot_data.index.values, current_plot_data.index.values)
detail_plot_stats = widgets.HTML(
    value="Empty <b>Empty</b>",
    placeholder='Stats',
    description='Stats',
)
detail_plot_stats.value = 'Empty'

######################### Parameter Selection and Detail Plot Updater #########################################

#timezone slider to cope with ipywidgets/bqplot issue
tz_slider = widgets.IntSlider(
    value=-4,
    min=-6,
    max=6,
    step=1,
    description='TimeZone:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

tz_msg = widgets.Label('Adjust TZ Slider if Zoom Graph does not work')

def data_slicer(data_stream, slice_start_time, slice_end_time):
    #helper function to return the sliced data
    #if start/stop outside range, return full data
    #data_stream is a pandas df
    # start/stop times are datetime64 values
    #the reason to have the selected_slice is growth capability
    #in the future, I want to maybe use these times to slice a movie
    #therefore I cannot store/lookup the index of the data. Need times.
    
    if slice_start_time < data_stream.index[0]:
        slice_start_time = data_stream.index[0]
    if (slice_end_time > data_stream.index[-1]) or (slice_end_time <= slice_start_time) :
        slice_end_time = data_stream.index[-1]
    slice_start_index = data_stream.index.searchsorted(slice_start_time)
    slice_end_index = data_stream.index.searchsorted(slice_end_time)
    sliced_data = data_stream.iloc[slice_start_index:slice_end_index]
    selected_slice = [sliced_data.index.values[0], sliced_data.index.values[-1]]
    return sliced_data, selected_slice


def detail_plot_update_brush(*args):
    #this call back needs to:
    # get the selected TS from the brush
    # update the detail plot with the "zoomed-in" data
    
    global current_plot_data, selected_slice, sliced_data, detail_plot
    ##### below, INSTEAD OF tz_slider, IT WOULD BE NICE IF THE TIMEZONE DETECTION WAS AUTOMATIC
    if slice_plot.brushintsel.selected is not None:
        if slice_plot.brushintsel.selected.size != 0:
            #conversion to pd.to_datetime necessary for python 3.6 compat
            #if I use np.datetime64(slice_plot.brushintsel.selected[0]), I get a deprecation error
            #so I need to go to pd.to_datetime and then back to np.datetime64.
            #what a mess...
            slice_start_time = np.datetime64(pd.to_datetime(slice_plot.brushintsel.selected[0]) - np.timedelta64((tz_slider.value-1)*-1, 'h'))
            slice_end_time = np.datetime64(pd.to_datetime(slice_plot.brushintsel.selected[-1]) - np.timedelta64((tz_slider.value-1)*-1, 'h'))


            sliced_data, selected_slice = data_slicer(current_plot_data, slice_start_time, slice_end_time)

            detail_plot.line.x = sliced_data.index.values
            detail_plot.line.y = np.transpose(sliced_data)

            detail_plot_stats.value = sliced_data.describe().to_html()
            slicebox.update_values(slice_start_time, slice_end_time)

# link brush to slice plot
slice_plot.brushintsel.observe(detail_plot_update_brush, 'selected')



def selected_index_changed(change):
    '''
    This callback treats the parameter selection/deselection on the map
    Depending on the active tab, it will also callback the appropriate graph updater
    '''
    global current_plot_data, sub_sampled_data
    selected_tab = FTAT_tabs.selected_index
    if (selected_tab == 0 or selected_tab ==2):
        #this is for time slice or analysis tabs
        #define the current plot data per selected parameters
        if len(slicemap.map.selected) != 0:
            selected_parameters_plot_data = np.transpose(sub_sampled_data[slicemap.map.selected].values)
            current_plot_data = sub_sampled_data[slicemap.map.selected]
    if selected_tab == 0:
        #this is the time slice tab
        if len(slicemap.map.selected) != 0:
            slice_plot.line.y = selected_parameters_plot_data
            slice_plot.fig.title = str(slicemap.map.selected)
            detail_plot.line.y = selected_parameters_plot_data
            detail_plot.fig.title = str(slicemap.map.selected) + ' - Zoomed'
    elif selected_tab == 1:
        #this is the save to disk tab
        pass
    elif selected_tab == 2:
        #this is the analysis tab
        if len(slicemap.map.selected) != 0:
            analysis_plot.line.y = selected_parameters_plot_data
            analysis_plot.y_data = selected_parameters_plot_data  ##
            analysis_plot.fig.title = str(slicemap.map.selected)
            analysis_plot.ys.min = selected_parameters_plot_data.min()
            analysis_plot.ys.max = selected_parameters_plot_data.max()
    elif selected_tab == 3:
        pass
    else:
        pass

# link selection handler to parameter map
slicemap.map.observe(selected_index_changed, 'selected')

# instantiate the slice selector
slicebox = sliceSelectDialog(current_plot_data)



############################### Plot Button Logic #########################
def on_plot_button_clicked(b):
    global current_plot_data, selected_slice, sliced_data
    slice_start_time = current_plot_data.index[0]
    slice_start_time = slice_start_time.replace(hour=slicebox.start_hour_box.value, 
                                                minute=slicebox.start_minute_box.value,
                                                second=slicebox.start_second_box.value)

    slice_end_time = current_plot_data.index[0]
    slice_end_time = slice_start_time.replace(hour=slicebox.end_hour_box.value, 
                                              minute=slicebox.end_minute_box.value,
                                              second=slicebox.end_second_box.value)
   
    sliced_data, selected_slice = data_slicer(current_plot_data, slice_start_time, slice_end_time)
    
    detail_plot.line.x = sliced_data.index.values
    detail_plot.line.y = np.transpose(sliced_data)
    
    detail_plot_stats.value = sliced_data.describe().to_html()

plot_button = widgets.Button(description='Plot Slice Below')
plot_button.on_click(on_plot_button_clicked)


########################## Save Slice Button and Logic ##############################

# time slices are saved to a dictionary for later analysis
TP_number_box = widgets.FloatText(
    value=0.0,
    description='TP#:',
    disabled=False,
    layout=Layout(width='130px')
)

TP_desc_box = widgets.Text(
    value='',
    placeholder='Type something',
    description='TP Title:',
    disabled=False
)
TS_message = Label()


################## Data Slice Selector Instance ####################
data_slice_selec = DataSliceSelect(time_slices_db)



################## Add/Remove Time Slices to db Logic #######################

def update_TP_saved_display(time_slices_db):
    display_text = 'TP#  Description \n'
    for key in time_slices_db:
        display_text = display_text  + str(key) + '  ' + time_slices_db[key][0] + '\n'
    return display_text

def on_saveTS_button_clicked(b):
    global time_slices_db
    if str(TP_number_box.value) in time_slices_db:
        TS_message.value = 'TP# {} already in db, choose a different one'.format(TP_number_box.value)
    else:
        TS_message.value = 'added TP {}'.format(TP_number_box.value)
        if detail_plot.xs.min:  # if the scale is None, go to ELSE!
            time_slices_db[str(TP_number_box.value)] = [TP_desc_box.value, np.datetime64(detail_plot.xs.min)
                                                        , np.datetime64(detail_plot.xs.max)]
        else:
            time_slices_db[str(TP_number_box.value)] = [TP_desc_box.value, selected_slice[0], selected_slice[-1]]
        TP_saved_dd.options = list(dict.keys(time_slices_db))
        time_slices_db_radio.options = list(dict.keys(time_slices_db))
        
def on_delTS_button_clicked(b):
    global time_slices_db
    if TP_saved_dd.value in time_slices_db:
        TS_message.value = 'Deleted'
        del time_slices_db[TP_saved_dd.value]
        TP_saved_dd.options = list(dict.keys(time_slices_db))
        time_slices_db_radio.options = list(dict.keys(time_slices_db))
    else:
        TS_message.value = 'trying to delete nothing?'


TP_saved_dd = widgets.Dropdown(
    options=time_slices_db,
    description='TP#:',
    disabled=False,
)


def on_TP_saved_dd_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        TS_message.value = ''

TP_saved_dd.observe(on_TP_saved_dd_change)
    
    
saveTS_button = widgets.Button(description='Save TSlice')
saveTS_button.on_click(on_saveTS_button_clicked)
delTS_button = widgets.Button(description='Delete TSlice')
delTS_button.on_click(on_delTS_button_clicked)

################## TS Radio Buttons ###################################################

time_slices_db_radio = widgets.RadioButtons(
    options=[],
    description='Slices:',
    disabled=False
)

def on_click_time_slices_db_radio(change):
    #everytime we select a slice, the analysis plot needs to be updated
    #using the parameters selected on the map and the TS from the dictionary
    global poly_order, current_plot_data, time_slices_db
    if (change['type'] == 'change') and (change['name'] == 'value') and (len(slicemap.map.selected) != 0) and (change['new'] != None):
        analysis_plot.x_data_slice_min = time_slices_db[change['new']][1]
        analysis_plot.x_data_slice_max = time_slices_db[change['new']][2]
        analysis_plot.update_plot(current_plot_data, slicemap.map.selected, time_slices_db[change['new']][1],
                                  time_slices_db[change['new']][2], poly_order)

time_slices_db_radio.observe(on_click_time_slices_db_radio, 'value')


################## Save Slices to Disk Logic ################################################
def save_slices(tp_dict):
    
    for key in tp_dict.keys():
        current_value = tp_dict[key]
        #use this for local tests
        #filepath = '~/Documents/'
        #use this one for jupyterlab
        filepath = '~/work/'
        filename = filepath + 'TP_' +  str(key) + '_' + current_value[0] + '.csv'
        #filename = f.path.split('.')[0] + '_TP_' +  str(key) + '_' + current_value[0] + '.csv'
        #think where we can save this so the user can retrieve later...
        #remember we are in a container.
        slice_start = current_value[1]
        slice_end = current_value[2]
        sliced_data = raw_data.iloc[(raw_data.index >= slice_start) & 
                                           (raw_data.index <= slice_end)]
        if save_all_parameters.value:
            sliced_data.to_csv(filename)
        else:
            sliced_data[slicemap.map.selected].to_csv(filename)
    # zipping it all
    zip_filename = filepath+'TP_zipped.zip'
    zip_file_wildcard = filepath+'TP*.csv'
    !rm $zip_filename
    !zip $zip_filename $zip_file_wildcard
    !rm $zip_file_wildcard
    save_feedback.value = 'files saved to disk'
        
def on_save_slices_button_clicked(b):
    save_slices(time_slices_db)



save_instructions = widgets.Label(
    value='Select parameters to save or select checkbox below to save all',
    disable=False
    )

save_feedback = widgets.Label(
    value='',
    disable=False
    )

save_all_parameters = widgets.Checkbox(
    value=False,
    description='save ALL PARAMETERS',
    disabled=False
)

save_slices_button = widgets.Button(description='Save Slices to Disk')
save_slices_button.on_click(on_save_slices_button_clicked)



################# Analysis Plot Definition #####################

analysis_plot = AnalysisPlot(current_plot_data.index.values, current_plot_data.index.values, time_slices_db)
poly_order = 1

analysis_poly_order_dd = widgets.Dropdown(
            options=[1,2,3,4],
            description='Poly Deg',
            value=1,
            disabled=False
        )

def on_change_analysis_poly_order_dd(change):
    global poly_order, current_plot_data
    if change['type'] == 'change' and change['name'] == 'value' and len(slicemap.map.selected) != 0:
        poly_order = change['new']
        #print(poly_order)
        analysis_plot.update_plot(current_plot_data, slicemap.map.selected, analysis_plot.xs.min, analysis_plot.xs.max, poly_order)
        #current_plot_data, parameter_list, slice_start, slice_end, poly_degree
            
analysis_poly_order_dd.observe(on_change_analysis_poly_order_dd)

analysis_plot_zoom_slider = SimpleZoomSlider(analysis_plot)

def on_analysis_zoom_slider_change(change):
    analysis_plot_zoom_slider.updateScale(analysis_plot)
    analysis_plot.update_plot(current_plot_data, slicemap.map.selected, 
                              np.datetime64(analysis_plot.xs.min),
                              np.datetime64(analysis_plot.xs.max),
                              poly_order)

analysis_plot_zoom_slider.zoom_slider.observe(on_analysis_zoom_slider_change, names='value')

# slice trim to zoomed figure logic
def slice_trim(tp_dict, key_to_find, slice_start, slice_end):
    for key in tp_dict.keys():
        if key == key_to_find:
            current_value = tp_dict[key]
            current_value[1] = slice_start
            current_value[2] = slice_end
            tp_dict[key] = current_value


def on_slice_trim_button_clicked(b):
    slice_trim(time_slices_db, time_slices_db_radio.value, np.datetime64(analysis_plot.xs.min), np.datetime64(analysis_plot.xs.max))
    
slice_trim_button = widgets.Button(description='Trim Current Slice')
slice_trim_button.on_click(on_slice_trim_button_clicked)



################# Strip Charts Definition #####################
strip_chart_button = widgets.Button(description='Plot Strip Chart Below')


strip_chart = StripChart(sub_sampled_data, slicemap, time_slices_db, time_slices_db_radio)


#####################################################################################################
# MAIN TABS DEFINITIONS
#####################################################################################################

tab_contents = ['Slice', 'Save', 'Analysis', 'StripChart']
text_children = [widgets.Text(description=name) for name in tab_contents]

slice_tab_children = VBox([slice_plot.fig, 
                           slicebox.slice_box,
                           plot_button,
                           detail_plot.fig, 
                           VBox([TS_message, HBox([saveTS_button, TP_number_box, TP_desc_box, TP_saved_dd, delTS_button])])
                          ]) 

save_tab_children = VBox([save_instructions, 
                          save_all_parameters, 
                          save_slices_button, 
                          save_feedback])

plot_tab_children = VBox([analysis_poly_order_dd, 
                          HBox([analysis_plot.fig, analysis_plot.fit_statistics]), 
                          analysis_plot_zoom_slider.zoom_slider, 
                          slice_trim_button])

strip_chart_children = strip_chart.widget()

complete_tabs_children = [slice_tab_children, save_tab_children, plot_tab_children, strip_chart_children]
FTAT_tabs = widgets.Tab()
FTAT_tabs.children = complete_tabs_children
for i in range(len(complete_tabs_children)):
    FTAT_tabs.set_title(i, tab_contents[i])
FTAT_tabs.layout = Layout(width='100%')

# linking parameter and tab selections
# callbacks

def update_logic(param01):
    selected_tab = FTAT_tabs.selected_index
    #reset the saved files feedback everytime the tabs as changed
    save_feedback.value = ''
    if selected_tab == 0:
        selected_index_changed(None)
    elif selected_tab == 1:
        pass
    elif selected_tab == 2:
        selected_index_changed(None)
    elif selected_tab == 3:
        pass
    else :
        pass
       

def on_FTAT_tabs_change(change):
    #this is just a wrapper function that will allow me to pass other parameters/objects
    #for now, just call the updater
    update_logic(1)
    
FTAT_tabs.observe(on_FTAT_tabs_change)

HBox([VBox([Box(children = [slicemap.map], layout=Layout(width='80%')), tz_msg, tz_slider, time_slices_db_radio]), 
      Box(children = [FTAT_tabs], layout=Layout(width='80%'))])